In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob

In [2]:
face_cascade=cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [3]:
#for data training of without mask
'''capture=cv2.VideoCapture(0)
data=[]
dat=[]
while True:
    flag,img=capture.read()
    if flag:
        faces=face_cascade.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),4)
            face=img[y:y+h,x:x+w,:]
            face=cv2.resize(face,(100,100))
            #print(len(data))
            if len(data)<400:
                data.append(face)
        cv2.imshow('trainer',img)
        if cv2.waitKey(2)==27 or len(data)>=400: break
capture.release()
cv2.destroyAllWindows()
np.save('without_mask.npy',data)'''

"capture=cv2.VideoCapture(0)\ndata=[]\ndat=[]\nwhile True:\n    flag,img=capture.read()\n    if flag:\n        faces=face_cascade.detectMultiScale(img)\n        for x,y,w,h in faces:\n            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),4)\n            face=img[y:y+h,x:x+w,:]\n            face=cv2.resize(face,(100,100))\n            #print(len(data))\n            if len(data)<400:\n                data.append(face)\n        cv2.imshow('trainer',img)\n        if cv2.waitKey(2)==27 or len(data)>=400: break\ncapture.release()\ncv2.destroyAllWindows()\nnp.save('without_mask.npy',data)"

In [4]:
#for data training of with mask
'''capture=cv2.VideoCapture(0)
data=[]
while True:
    flag,img=capture.read()
    if flag:
        faces=face_cascade.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),4)
            face=img[y:y+h,x:x+w,:]
            face=cv2.resize(face,(100,100))
            #print(len(data))
            if len(data)<400:
                data.append(face)
        cv2.imshow('trainer',img)
        if cv2.waitKey(2)==27 or len(data)>=400: break
capture.release()
cv2.destroyAllWindows()
'''

"capture=cv2.VideoCapture(0)\ndata=[]\nwhile True:\n    flag,img=capture.read()\n    if flag:\n        faces=face_cascade.detectMultiScale(img)\n        for x,y,w,h in faces:\n            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),4)\n            face=img[y:y+h,x:x+w,:]\n            face=cv2.resize(face,(100,100))\n            #print(len(data))\n            if len(data)<400:\n                data.append(face)\n        cv2.imshow('trainer',img)\n        if cv2.waitKey(2)==27 or len(data)>=400: break\ncapture.release()\ncv2.destroyAllWindows()\n"

In [5]:
path=glob.glob("C:/Users/pytho/Desktop/Mask/*.jpg")
data=[]
for file in path:
    img=cv2.imread(file)
    faces=face_cascade.detectMultiScale(img)
    for x,y,w,h in faces:
        face=img[y:y+h,x:x+w,:]
        face=cv2.resize(face,(100,100))
        data.append(face)
    #cv2.imshow('trainer',img)
    #cv2.waitKey(2)==27: break
#capture.release()


In [6]:
np.save('with_mask.npy',data)

In [7]:
#path=glob.glob("C:/Users/pytho/Desktop/Mask/*.jpg")
path=glob.glob("C:/Users/pytho/Desktop/Non Mask/*.jpg")
data=[]
for file in path:
    img=cv2.imread(file)
    faces=face_cascade.detectMultiScale(img)
    for x,y,w,h in faces:
        face=img[y:y+h,x:x+w,:]
        face=cv2.resize(face,(100,100))
        data.append(face)
np.save('without_mask.npy',data)

In [8]:
no_mask=np.load('without_mask.npy')
mask=np.load('with_mask.npy')

In [9]:
no_mask.shape

(318, 100, 100, 3)

In [10]:
mask.shape

(379, 100, 100, 3)

In [11]:
mask=mask.reshape(379,100*100*3)
no_mask=no_mask.reshape(318,100*100*3)

In [12]:
no_mask.shape

(318, 30000)

In [13]:
x=np.r_[no_mask,mask]
x.shape[0]

697

In [14]:
y=np.ones(x.shape[0])
y[:348]=0.0

In [15]:
d={1:'Mask',0:'No mask'}

In [16]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [17]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30)
x_train.shape

(487, 30000)

In [18]:
m=SVC()
m.fit(x_train,y_train)
y_pred=m.predict(x_test)
accuracy_score(y_test,y_pred)

0.8523809523809524

In [19]:
haar_data=cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade=cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_eye.xml')

In [35]:
cap=cv2.VideoCapture(r'Videos/AA.mp4')
#cap=cv2.VideoCapture(r'Downloads/fucking_awesome.mp4')
data=[]
lol=1
font=cv2.FONT_HERSHEY_COMPLEX
while (cap.isOpened()):
    flag,img=cap.read()
    img=cv2.resize(img,(700,500))
    if flag:
        faces=haar_data.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),4)
            face=img[y:y+h,x:x+w,:]
            face=cv2.resize(face,(100,100))
            face=face.reshape(1,-1)
            pred=m.predict(face)[0]
            n=d[int(pred)]
            cv2.putText(img,n,(x,y),font,1,(0,0,255),2)
            #print(n)
        cv2.imshow('hell0_world',img)
        if cv2.waitKey(lol) & 0xFF==27:
            break

cap.release()
cv2.destroyAllWindows()